In [ ]:
!cd ../../ && python3 setup.py build_ext --inplace
%load_ext autoreload
%autoreload 2

import plotly.express as px
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# Initialize the Plotly notebook mode
init_notebook_mode(connected=True)

In [ ]:
import numpy as np
from random import random
from numpy.random import randint
from scipy.ndimage import gaussian_filter
from nanopyx.core.transform.binning import rebin2d
from nanopyx.core.transform.noise.randomNoise import addMixedGaussianPoissonNoise, addPerlinNoise, addRamp

# generate some random ground truth
w = 1280
h = 1280
image_gt = np.ones((w, h), dtype="float32") * 1000
addPerlinNoise(image_gt, amp=1000, f=100, octaves=3)
    
image_ref = gaussian_filter(image_gt, 15)
image_ref = rebin2d(image_ref, 10, mode="mean")
addRamp(image_ref, 1000)
addMixedGaussianPoissonNoise(image_ref, 10, 10)  

image_sr = image_gt.copy()
image_sr = gaussian_filter(image_sr, 3)
image_sr = rebin2d(image_sr, 10, mode="mean")

fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=('image_gt','image_ref', 'image_sr'))
fig.add_trace(go.Heatmap(z=image_gt), 1, 1)
fig.add_trace(go.Heatmap(z=image_ref), 1, 2)   
fig.add_trace(go.Heatmap(z=image_sr), 1, 3)    


In [ ]:
from nanopyx.core.sr.errorMap import ErrorMap

squirrelErrorMap = ErrorMap()
squirrelErrorMap.optimise(image_ref, image_sr)

px.imshow(squirrelErrorMap.imRSE)